In [12]:
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import pandas as pd
import os
import requests
import time
from scipy.stats import linregress

In [13]:
data=pd.read_csv("data_cities.csv")

In [11]:
gmaps.configure(api_key='')

In [4]:
figure_layout={'width':'450px',
               'height':'450px', 
               'border':'3px', 
               'padding':'3px'
            }
gmaps.figure(layout=figure_layout)

Figure(layout=FigureLayout(border='3px', height='450px', padding='3px', width='450px'))

In [5]:
locations=data[['Lat', 'Lng']]
weight=data['Humidity']
fig=gmaps.figure(layout=figure_layout, map_type='HYBRID')
heatmap_layer=gmaps.heatmap_layer(locations, weights=weight, max_intensity=2, dissipating=True,
                                  point_radius=3,  opacity=0.6,  gradient=['white', 'gray'])
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='3px', height='450px', padding='3px', width='450px'))

# Create new DataFrame fitting weather criteria

In [6]:
#•	Narrow down the cities to fit weather conditions.
#•	Drop any rows will null values.
narrowed_city_df = data.loc[(data["Max Temp"] > 70) & (data["Max Temp"] < 80) & (data["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06


# Hotel map

In [10]:
hotellist = []
g_key=''
for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [9]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='3px', height='450px', padding='3px', width='450px'))